In [7]:
import tensorflow as tf
import numpy as np
import scipy

import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob, os, sys, gc
import pickle

sys.path.insert(0,'..') # add parent directory to path
from prior_attention import build_prior_attentions
import utils
from featureextractor import FeatureExtractor
from config import Config

# this is important to make sure the Config is reloaded when modified
%load_ext autoreload
%autoreload 2
# %pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dict(vars(Config))

{'__module__': 'config',
 'main_path': '/home/mikey/Data/POETdataset/PascalImages',
 'class_names': ['dog',
  'aeroplane',
  'boat',
  'bicycle',
  'cat',
  'cow',
  'diningtable',
  'horse',
  'motorbike',
  'sofa'],
 'IMG_SIZE': 480,
 'PATCH_WIDTH': 120,
 'SMALLER_IMG_SIZE': 150,
 'T': 16,
 'new_dir': 'soft_attention_features_120',
 'new_dir_img': 'soft_attention_images_120',
 'name_to_class_dict': {'dog': 0,
  'aeroplane': 1,
  'boat': 2,
  'bicycle': 3,
  'cat': 4,
  'cow': 5,
  'diningtable': 6,
  'horse': 7,
  'motorbike': 8,
  'sofa': 9},
 'train_ids': ['dog_2010_002398.jpg',
  'horse_2009_005161.jpg',
  'cat_2010_002662.jpg',
  'cat_2010_003687.jpg',
  'aeroplane_2008_006677.jpg',
  'cat_2010_000389.jpg',
  'bicycle_2009_003187.jpg',
  'motorbike_2010_005366.jpg',
  'dog_2010_000276.jpg',
  'dog_2010_005330.jpg',
  'aeroplane_2008_005905.jpg',
  'aeroplane_2008_008179.jpg',
  'cat_2008_003772.jpg',
  'cat_2009_004477.jpg',
  'motorbike_2011_000397.jpg',
  'cat_2010_001939.jpg',

In [11]:
# extract features for a list of ids
def extract_features(name_to_class_dict, main_path, new_dir, model, img_size, patch_width, ids, mode):
    
    if os.path.exists(new_dir):
        print("Directory", new_dir, "already exists. Exitting...")
#         return
    else:
        os.makedirs(new_dir)
    
    print("Extracting features in mode", mode)
    
    try:
        for class_name, class_number in Config.name_to_class_dict.items():
            files = []
            class_ids = []
            print(class_name)
            for i, file_name in enumerate(ids):
                if class_name in file_name:
                    files.append(main_path + os.path.sep + file_name)
                    class_ids.append(file_name)

            features = FeatureExtractor.get_features(files, model, img_size, patch_width)

            utils.save_object(features, new_dir + os.path.sep + class_name + '_'+mode+'_features.pkl')
            utils.save_object(class_ids, new_dir + os.path.sep + class_name + '_'+mode+'_names.pkl')
    except:
        os.rmdir(new_dir)
    
# save resized images to fit in memory when loading
def save_images(name_to_class_dict, main_path, new_dir_img, img_size, ids, mode):
    
    if os.path.exists(new_dir_img):
        print("Directory", new_dir_img, "already exists. Exitting...")
#         return
    else:
        os.makedirs(new_dir_img)
    
    print("Saving images in mode", mode)
        
    try:
        for class_name, class_number in name_to_class_dict.items():
            files = []
            print(class_name)
            for i, file_name in enumerate(ids):
                if class_name in file_name:
                    files.append(main_path + os.path.sep + file_name)

            images = FeatureExtractor._load_images(files, img_size=img_size)
            print(images.shape)
            save_path = new_dir_img + os.path.sep + class_name + '_' + mode+'.pkl'
            print("Saving...", save_path)
            utils.save_object(images, save_path)
    except:
        os.rmdir(new_dir_img)

# This code above should check if the pickle file already exists not if the directory exists

In [4]:
model = FeatureExtractor.get_feature_extraction_model(img_size = Config.PATCH_WIDTH)

In [12]:
# load from config
feature_extractor_args = [Config.name_to_class_dict, Config.main_path, Config.new_dir, model, 
                          Config.IMG_SIZE, Config.PATCH_WIDTH]
image_saving_args = [Config.name_to_class_dict, Config.main_path, Config.new_dir_img, 
                      Config.SMALLER_IMG_SIZE]

# extract features for patches
extract_features(*feature_extractor_args, mode='train', ids=Config.train_ids)
extract_features(*feature_extractor_args, mode='test', ids=Config.test_ids)

# save smaller images for visualization
save_images(*image_saving_args, mode='train', ids=Config.train_ids)
save_images(*image_saving_args, mode='test', ids=Config.test_ids)

Directory soft_attention_features_120 already exists. Exitting...
Extracting features in mode train
dog
(1128, 4, 4, 120, 120, 3)
18048/18048 [==============================] - 23s 1ms/step
(18048, 2048)
aeroplane
(599, 4, 4, 120, 120, 3)
9584/9584 [==============================] - 12s 1ms/step
(9584, 2048)
boat
(454, 4, 4, 120, 120, 3)
7264/7264 [==============================] - 9s 1ms/step
(7264, 2048)
bicycle
(480, 4, 4, 120, 120, 3)
7680/7680 [==============================] - 10s 1ms/step
(7680, 2048)
cat
(943, 4, 4, 120, 120, 3)
15088/15088 [==============================] - 19s 1ms/step
(15088, 2048)
cow
(265, 4, 4, 120, 120, 3)
4240/4240 [==============================] - 5s 1ms/step
(4240, 2048)
diningtable
(427, 4, 4, 120, 120, 3)
6832/6832 [==============================] - 9s 1ms/step
(6832, 2048)
horse
(428, 4, 4, 120, 120, 3)
6848/6848 [==============================] - 9s 1ms/step
(6848, 2048)
motorbike
(449, 4, 4, 120, 120, 3)
7184/7184 [==============================